# LAB-GPU EJERCICIO EXTRA

### Librerias

In [1]:
import numpy as np
import cupy as cp
from multiprocessing import Pool
import time
from numba import float32, cuda
from IPython import get_ipython

### Funciones

#### Función original

In [2]:
def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

#### Funciones multiprocessing

In [3]:
def parallel_reduction(X, num_processes):
    """Perform parallel reduction using multiprocessing with num_processes."""
    n = len(X)
    chunk_size = n // num_processes
    sub_arrays = [X[i * chunk_size: (i + 1) * chunk_size] for i in range(num_processes - 1)]
    sub_arrays.append(X[(num_processes - 1) * chunk_size:])  # Add the last chunk
    
    with Pool(processes=num_processes) as pool:
        results = pool.map(reduc_operation, sub_arrays)
    
    return sum(results)

#### Funciones Cupy

In [4]:
def sum_with_cupy(array_cpu):
    """
    Realiza la suma de los elementos de un array en la GPU utilizando Cupy.
    Incluye transferencia de datos entre CPU y GPU.
    """
    array_gpu = cp.array(array_cpu)
    result_gpu = cp.sum(array_gpu)
    result_cpu = result_gpu.item()

    return result_cpu, elapsed_time


#### Funciones Numba

In [22]:
@cuda.reduce
def reduc_operation_cuda(x, y):
    """
    Realiza la suma de dos elementos. 
    Esta operación será utilizada por el decorador @cuda.reduce para realizar la reducción.
    """
    return x + y
    
def sum_with_numba_gpu(arr):
    """
    Realiza la suma de los elementos de un array ya alojado en la GPU utilizando Numba y @cuda.reduce.
    """
    arr = arr.astype(np.float32)
    d_arr = cuda.to_device(arr)
    return np.sum(d_arr.copy_to_host())
    

### Variables

In [6]:
# Definir la variable value

value = None

# Intentar obtener el valor de la variable VALUE pasada como un argumento

try:
    # Obtener el valor de la variable VALUE definida en el entorno
    ipython = get_ipython()
    value = int(ipython.getoutput("echo $VALUE")[0].strip())
    if value <= 0:
        raise ValueError("El valor de VALUE debe ser un entero positivo.")
except IndexError:
    print("\nNo se proporcionó VALUE. Usando el valor por defecto: 10^6")
    value = 10**6
except ValueError as e:
    print(f"\nError en el valor de VALUE: {e}. Usando el valor por defecto: 10^6")
    value = 10**6

print(f"\nRunning with {value} elements.")

# Crear los arrays

X = np.arange(value, dtype=np.float64)  # Array en la CPU
Y = cp.arange(value, dtype=cp.float64)  # Array en la GPU



Error en el valor de VALUE: invalid literal for int() with base 10: ''. Usando el valor por defecto: 10^6

Running with 1000000 elements.


### Código Inicial

In [7]:
# Utilizando en secuencial

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("\n Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")

# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")

# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum(), "\n")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")


 Time taken by reduction operation using a function: 97.4 ms ± 1.16 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 499999500000.0

-----------------------------------------------------------------------------------------------------------------------------------------------

Time taken by reduction operation using numpy.sum(): 201 µs ± 1.68 µs per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Now, the result using numpy.sum(): 499999500000.0 
 
-----------------------------------------------------------------------------------------------------------------------------------------------

Time taken by reduction operation using numpy.ndarray.sum(): 195 µs ± 104 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 499999500000.0 

-----------------------------------------------------------------------------------------------------------------------------------

### Código Nuevo 

#### Utilizando multiprocessing

In [8]:
print("\nSuma utilizando multiprocessing:")

num_processes_list = [1, 2, 4]

for num_processes in num_processes_list:
    start_time = time.time()
    result = parallel_reduction(X, num_processes)
    elapsed_time = time.time() - start_time
    print(f"\n Time taken by reduction operation using {num_processes} process(es):", elapsed_time)
    print(f"And the result of the sum using {num_processes} process(es) is: {result}\n")
    print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")


Suma utilizando multiprocessing:

 Time taken by reduction operation using 1 process(es): 0.13155746459960938
And the result of the sum using 1 process(es) is: 499999500000.0

-----------------------------------------------------------------------------------------------------------------------------------------------


 Time taken by reduction operation using 2 process(es): 0.07447576522827148
And the result of the sum using 2 process(es) is: 499999500000.0

-----------------------------------------------------------------------------------------------------------------------------------------------


 Time taken by reduction operation using 4 process(es): 0.05488848686218262
And the result of the sum using 4 process(es) is: 499999500000.0

-----------------------------------------------------------------------------------------------------------------------------------------------



#### Utilizando Cupy

In [9]:
print("\nSuma utilizando Cupy:")

result, elapsed_time = sum_with_cupy(X)

# Imprimir resultados
print("\n Time taken by reduction operation using Cupy:")
print(f"Resultado de la suma en la GPU (devuelto a la CPU): {result}")
print(f"Tiempo total (incluyendo transferencia de datos): {elapsed_time:.6f} segundos\n")
print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")



Suma utilizando Cupy:

 Time taken by reduction operation using Cupy:
Resultado de la suma en la GPU (devuelto a la CPU): 499999500000.0
Tiempo total (incluyendo transferencia de datos): 0.054888 segundos

-----------------------------------------------------------------------------------------------------------------------------------------------



#### Utilizando Numba

In [23]:
print("\nSuma utilizando una ufunc personalizada:")

start_time = time.time()
result = sum_with_numba_gpu(X)
elapsed_time = time.time() - start_time

print("\n Time taken by reduction operation using Numba @cuda.reduce:")
print(f"Resultado de la suma calculado en la GPU: {result}")
print(f"Tiempo total: {elapsed_time:.6f} segundos\n")
print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")


Suma utilizando una ufunc personalizada:

 Time taken by reduction operation using Numba @cuda.reduce:
Resultado de la suma calculado en la GPU: 499999801344.0
Tiempo total: 0.025991 segundos

-----------------------------------------------------------------------------------------------------------------------------------------------



### Análisis Comparativo: Cupy y Numba en la GPU

En esta práctica, se realizaron comparaciones entre los paquetes **Cupy** y **Numba** para realizar una operación de suma en arrays grandes utilizando la GPU. Ambos enfoques fueron evaluados considerando la capacidad de cálculo paralelo de las GPUs y la facilidad de uso que ofrecen estas herramientas.

#### Observaciones sobre Cupy
**Cupy** es una biblioteca diseñada específicamente para GPUs, que replica la funcionalidad de **NumPy** pero optimizada para aprovechar al máximo el hardware de la GPU. Durante esta práctica, la función `cp.sum` se utilizó para calcular la suma directamente en la GPU. Gracias a las optimizaciones internas de **Cupy**, este enfoque fue eficiente y rápido, especialmente para operaciones estándar como la suma de arrays grandes.

#### Observaciones sobre Numba
Por otro lado, **Numba** permite definir funciones personalizadas que se ejecutan en la GPU mediante compilación JIT (*Just-In-Time*). Se implementó una ufunc para realizar la suma del array directamente en la GPU. Aunque **Numba** ofrece mayor flexibilidad y control sobre los cálculos, el rendimiento puede estar condicionado por la implementación de la función y la configuración del hardware. Este enfoque resulta útil para personalizar algoritmos o realizar operaciones más complejas que no están predefinidas.

#### Comparación de Rendimiento
- **Cupy** resultó ser más rápido para operaciones predefinidas como la suma, debido a su diseño optimizado y su integración nativa con GPUs.
- **Numba** se destacó por su capacidad de personalización, lo que es crucial para escenarios donde se requieren operaciones específicas o no estándar.

#### Conclusión
Ambos paquetes son herramientas valiosas para aprovechar el poder computacional de las GPUs:
- **Cupy** es ideal para operaciones estándar y predefinidas, como la suma, gracias a su simplicidad y velocidad.
- **Numba** es más adecuado para algoritmos personalizados o situaciones que requieren un control detallado sobre los cálculos.

La elección entre estos enfoques depende del tipo de tarea y los requisitos del proyecto.
